In [1]:
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.schema import Document
from langchain_community.chat_models import ChatOllama
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import os
import fitz

In [2]:
# Directorio para cargar los archivos PDF
pdf_directory = './doc'  # Ajusta la ruta según tu estructura de directorios
os.makedirs(pdf_directory, exist_ok=True)

# Función para verificar si un archivo es PDF
def is_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:
            # Verificar si el archivo empieza con la cabecera %PDF
            header = file.read(4)
            if header != b'%PDF':
                return False

        # Intentar abrir el archivo con PyMuPDF
        doc = fitz.open(file_path)
        doc.close()
        return True
    except (fitz.FitzError, IOError):
        return False

# Directorio de entrada que contiene archivos PDF
input_directory = './doc'  # Cambia esta ruta si es diferente

# Mover y validar los archivos cargados
valid_pdf_files = []
for filename in os.listdir(input_directory):
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(input_directory, filename)
        print(f"Cargando archivo: {file_path}")

        # Mover el archivo al directorio especificado
        destination_path = os.path.join(pdf_directory, filename)

        # Mover el archivo al directorio especificado
        os.rename(file_path, destination_path)

        # Validar si es un PDF
        if is_pdf(destination_path):
            valid_pdf_files.append(filename)
        else:
            print(f"{filename} no es un archivo PDF válido y será ignorado.")

# Imprimir archivos válidos
print("Archivos PDF válidos:", valid_pdf_files)

Cargando archivo: ./doc\preguntas.pdf
Cargando archivo: ./doc\respuestas.pdf
Cargando archivo: ./doc\teoria.pdf
Archivos PDF válidos: ['preguntas.pdf', 'respuestas.pdf', 'teoria.pdf']


In [3]:
# Directorio que contiene los archivos PDF
pdf_directory = './doc'

# Lista para almacenar todos los documentos
all_docs = []

# Iterar sobre todos los archivos en el directorio
for filename in os.listdir(pdf_directory):
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(pdf_directory, filename)
        print(f"Cargando archivo: {file_path}")

        try:
            # Cargar el PDF
            loader = PyPDFLoader(file_path)
            docs = loader.load()

            # Agregar los documentos cargados a la lista
            all_docs.extend(docs)
            print(f"Archivo {filename} cargado exitosamente.")
        except Exception as e:
            print(f"Error al cargar {filename}: {e}")

# Imprimir la cantidad total de partes del PDF
print(f"Cantidad total de partes del PDF: {len(all_docs)}")

Cargando archivo: ./doc\preguntas.pdf
Archivo preguntas.pdf cargado exitosamente.
Cargando archivo: ./doc\respuestas.pdf
Archivo respuestas.pdf cargado exitosamente.
Cargando archivo: ./doc\teoria.pdf
Archivo teoria.pdf cargado exitosamente.
Cantidad total de partes del PDF: 53


# RecursiveCharacterTextSplitter

In [4]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #modelo de embeddings gratis

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,           # Tamaño del fragmento
        chunk_overlap=0,         # Superposición entre fragmentos
        separators=['\n\n', '\n', ' ', '']  # Separadores para dividir el texto
    )

splits = text_splitter.split_documents(all_docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./vectordb')

retriever = vectorstore.as_retriever()

c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, a

In [5]:
llm = ChatOllama(model='llama3.1:latest', temperature=0)

In [6]:
system_prompt = (
    "Usted es un asistente para tareas de respuesta a preguntas. Utiliza los siguientes elementos del contexto recuperado para responder a la pregunta. Responde fielmente con la información pasada por el contexto.Si no conoce la respuesta, diga simplemente que necesitas para poder responder. Utiliza y procura que la respuesta sea concisa"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

chain = create_stuff_documents_chain(llm, prompt)
rag = create_retrieval_chain(retriever, chain)

In [7]:
results = rag.invoke({"input": "¿Cuál es la idea básica del proyecto?"})
results

{'input': '¿Cuál es la idea básica del proyecto?',
 'context': [Document(metadata={'page': 5, 'source': './doc\\teoria.pdf'}, page_content='las semejanzas y diferencias. Además, es conveniente analizar las \ncausas que motivaron el fracaso de otras ideas parecidas. Por \núltimo, es importante definir las novedades o ventajas competitivas \nque  aportará nuestro producto o servicio al mercado frente a la \ncompetencia. 2.1.1. Objetivo\nEl objetivo fundamental de este apartado es concretar la actividad \nde la empresa en cuanto a los productos y servicios a comercializar \nen el presente inmediato, quiénes son los clientes potenciales, el \námbito geográfico, así como lasventajas competitivas y capacidades \nsingulares. En definitiva, se trata de resumir los aspectos clave del \nnegocio que se quiere poner en marcha.\n2.1.2. Contenido\nPara explicar en qué va a consistir el negocio, debemos hacerlo de \nuna forma clara y concisa, evitando las generalidades y el lenguaje \nexcesivamente e

In [8]:
results['answer']

'Según el contexto proporcionado, la idea básica del proyecto parece ser desarrollar un plan de negocio que concreta la actividad de la empresa en cuanto a los productos y servicios a comercializar en el presente inmediato. Esto incluye identificar a los clientes potenciales, el ámbito geográfico, así como las ventajas competitivas y capacidades singulares del negocio.'


# SemanticChunker

In [9]:
from langchain_experimental.text_splitter import SemanticChunker


# Crear el modelo de embeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Crear el Semantic Chunker
semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")

semantic_chunks = semantic_chunker.create_documents([d.page_content for d in all_docs])

# Dividir los documentos en partes semánticas manejables
for semantic_chunk in semantic_chunks:
  if "Effect of Pre-training Tasks" in semantic_chunk.page_content:
    print(semantic_chunk.page_content)
    print(len(semantic_chunk.page_content))

# Crear el vector store y almacenar los vectores
vectorstore = Chroma.from_documents(semantic_chunks, embedding=embeddings, persist_directory='./vectordb2')

# Configurar el recuperador
retriever = vectorstore.as_retriever()

c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
llm = ChatOllama(model='llama3.1:latest', temperature=0)

In [11]:
system_prompt = (
    "Usted es un asistente para tareas de respuesta a preguntas. Utiliza los siguientes elementos del contexto recuperado para responder a la pregunta. Responde fielmente con la información pasada por el contexto.Si no conoce la respuesta, diga simplemente que necesitas para poder responder. Utiliza y procura que la respuesta sea concisa"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

chain = create_stuff_documents_chain(llm, prompt)
rag = create_retrieval_chain(retriever, chain)

In [12]:
results = rag.invoke({"input": "¿Cuál es la idea básica del proyecto?"})
results

{'input': '¿Cuál es la idea básica del proyecto?',
 'context': [Document(page_content='¿Qué es?'),
  Document(page_content='1. Presentación\ndel\nProyecto\n1.1. Descripción\ndel\nNegocio\nAyuda\npara\nResponder:\n●\nIdea\nBásica\ndel\nProyecto:\no\nPiensa\nen\nqué\noportunidad\nde\nmercado\nestás\naprovechando\ncon\ntu\nidea\nde \nnegocio. Según\nel\ndocumento,\nla\nidea\ndebe\ncubrir\nuna\ndemanda\nno \nsatisfecha\no\nmejorar\nun\nproducto/servicio\nexistente. o\nDescribe\nel\nproducto\no\nservicio\ndestacando\nsi\nes\nalgo\nnuevo\nen\nel \nmercado\no\nuna\ninnovación\nsobre\nalgo\nya\nexistente. ●\nProductos\no\nServicios:\no\nSi\nes\nun\nproducto,\ndescribe\nsus\ncaracterísticas,\nutilidad\ny\nsi\nes\nalgo \nnovedoso\no\nconocido. Explica\ncómo\nfuncionará\ny\ncuáles\nson\nsus\nventajas \ncompetitivas. o\nSi\nes\nun\nservicio,\ndetalla\nlo\nque\nse\nva\na\nhacer ,\ncómo\nse\nhará\ny\nqué \nbeneficios\naportará\nal\ncliente. ●\nRazones\npara\nCrear\nla\nEmpr esa:\no\nSegún\nel\ndocum

In [13]:
results['answer']

'No puedo determinar el proyecto que estás haciendo. ¿Puedes proporcionarme más información sobre lo que estás tratando de hacer?'